In [2]:
from ast import Tuple
import os
# from tkinter.font import names
from kubernetes import client, config
from typing import List
from my_policy_interface import NodeInfo, PodInfo
from node_delay_measure_ParallelComp import measure_http_latency
from node_bandwidth_measure_ParallelComp import measure_bandwidth

In [1]:
# import 
from my_cluster_utils import (
    gather_all_nodes,
    build_nodeinfo_objects,
    gather_all_pods,
    build_podinfo_objects
)

# import sys
# import os

# # Get the absolute path of the current file's folder
# current_dir = os.path.dirname(os.path.abspath("__file__"))

# # Construct the path to the three different policies folders
# policy1_graph_dynamics_path = os.path.join(current_dir, "..", "Policy1_eval_Graph_dynamics")
# policy2_latencyAware_path = os.path.join(current_dir, "..", "Policy2_eval_Delay_dynamics")
# policy3_BandwidthAware_path = os.path.join(current_dir, "..", "Policy3_eval_Bandwidth_dynamics")


# # Add that folder to the Python path
# sys.path.append(policy1_graph_dynamics_path)
# sys.path.append(policy2_latencyAware_path)  
# sys.path.append(policy3_BandwidthAware_path)

from Policy1_eval_Graph_dynamics import build_call_graph, Policy1CallGraphAware
from Policy2_eval_Delay_dynamics import Policy2LatencyAware
from Policy3_eval_Bandwidth_dynamics import Policy3BandwidthAware


# import Policy1CallGraphAware, Policy2LatencyAware, Policy3BandwidthAware

Pods already exist in the namespace. Skipping DaemonSet deployment.
{'k8s-worker-3': {'k8s-worker-3': None, 'k8s-worker-9': 3.543, 'k8s-worker-7': 3.5669999999999997, 'k8s-worker-5': 4.589, 'k8s-worker-1': 0.488, 'k8s-worker-6': 8.468, 'k8s-worker-4': 2.846, 'k8s-worker-2': 9.592, 'k8s-worker-8': 16.566000000000003}, 'k8s-worker-6': {'k8s-worker-6': None, 'k8s-worker-7': 13.559999999999999, 'k8s-worker-9': 15.613999999999999, 'k8s-worker-5': 10.609, 'k8s-worker-8': 10.693, 'k8s-worker-1': 7.556, 'k8s-worker-2': 23.793000000000003, 'k8s-worker-3': 8.53, 'k8s-worker-4': 18.703}, 'k8s-worker-9': {'k8s-worker-9': None, 'k8s-worker-3': 3.6350000000000002, 'k8s-worker-8': 7.651, 'k8s-worker-7': 11.54, 'k8s-worker-1': 3.525, 'k8s-worker-4': 15.854, 'k8s-worker-2': 11.499, 'k8s-worker-5': 19.759, 'k8s-worker-6': 15.57}, 'k8s-worker-7': {'k8s-worker-7': None, 'k8s-worker-1': 6.6, 'k8s-worker-4': 15.478, 'k8s-worker-5': 11.536, 'k8s-worker-9': 11.725, 'k8s-worker-8': 18.728, 'k8s-worker-3': 3.75

ModuleNotFoundError: No module named 'Policy1_eval_Graph_dynamics'

In [ ]:


def main():
    # 1) Gather cluster data
    raw_nodes = gather_all_nodes()
    candidate_nodes = build_nodeinfo_objects(raw_nodes)

    # Suppose your social-network microservices are in namespace "social-network"
    raw_pods = gather_all_pods(namespace="social-network")
    pods = build_podinfo_objects(raw_pods)

    # 2) (Optional) Build a call graph from 'graph_builder.py'
    #    This might be used by Policy1 to place heavily communicating microservices.
    G = build_call_graph(namespace="social-network")
    # Convert the NetworkX graph to a simple dict: (src, dst) -> traffic_kb
    traffic_matrix = {}
    for (u, v, data) in G.edges(data=True):
        traffic_matrix[(u, v)] = data.get("weight", 0.0)  # store as KB

    # 3) Initialize each policy with scenario-specific config
    policy1 = Policy1CallGraphAware()
    policy1.initialize_policy(config={"traffic_matrix": traffic_matrix})

    policy2 = Policy2LatencyAware()
    policy2.initialize_policy(config={"latency_threshold": 5.0})  # example

    policy3 = Policy3BandwidthAware()
    policy3.initialize_policy(config={"high_traffic_threshold": 300.0})  # example

    # 4) Choose which scenario you want to test
    # Scenario 1: call-graph dynamics => we expect policy1 to shine
    print("=== Scenario1: Call-Graph_Aware ===")
    decisions_s1 = policy1.schedule_all(pods, candidate_nodes)
    for dec in decisions_s1:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # Scenario 2: cross-node delays => policy2 might do better
    print("\n=== Scenario2: Latency_Aware ===")
    decisions_s2 = policy2.schedule_all(pods, candidate_nodes)
    for dec in decisions_s2:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # Scenario 3: cross-node bandwidth changes => policy3 might do better
    print("\n=== Scenario3: Bandwidth_Aware ===")
    decisions_s3 = policy3.schedule_all(pods, candidate_nodes)
    for dec in decisions_s3:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

if __name__ == "__main__":
    main()


In [1]:

# file: main_controller.py
# import sys
# sys.path.append('/home/ubuntu/iDynamics')
# from singlePodPolicy_NetMARKS import NetMARKS_Policy
# from MultiPodPolicy_TraDE import TraDE_Policy

from my_cluster_utils import gather_all_nodes, gather_all_pods, build_nodeinfo_objects, build_podinfo_objects
# import my_cluster_utils

from my_policy_interface import AbstractSchedulingPolicy

def main():
    # 1. Connect to cluster, prom, gather data
    raw_nodes = gather_all_nodes()  # returns raw K8s node objects
    raw_pods = gather_all_pods()    # returns raw K8s pod objects

    node_infos = build_nodeinfo_objects(raw_nodes)
    pod_infos  = build_podinfo_objects(raw_pods)

    # 2. Choose a policy
    # policy = NetMARKS_Policy()  # or TraDE_Policy
    # policy.initialize_policy({
    #     "prom_url": "http://10.105.116.175:9090", # Prometheus URL
    #     "namespace": "social-network",
    #     "timerange": 1,
    # })

    # # 3. Decide if we want single-pod or multi-pod scheduling
    # #    For NetMARKS, let's do single-pod:
    # target_pod = pod_infos[0]
    # decision = policy.schedule_pod(target_pod, node_infos)
    # print(f"Schedule {decision.pod_name} on {decision.selected_node}")

    # 4. Patch the cluster, wait for rolling updates, etc.
    # if decision.selected_node != "NoSuitableNode":
    #     patch_deployment_to_node(target_pod.deployment_name, decision.selected_node)

if __name__ == "__main__":
    main()


Pods already exist in the namespace. Skipping DaemonSet deployment.
{'k8s-worker-7': {'k8s-worker-7': None, 'k8s-worker-1': 6.584, 'k8s-worker-4': 15.524, 'k8s-worker-9': 11.658, 'k8s-worker-3': 3.489, 'k8s-worker-5': 11.446, 'k8s-worker-8': 18.970000000000002, 'k8s-worker-2': 16.679, 'k8s-worker-6': 13.632}, 'k8s-worker-6': {'k8s-worker-6': None, 'k8s-worker-8': 10.815999999999999, 'k8s-worker-5': 10.496, 'k8s-worker-9': 15.6, 'k8s-worker-1': 7.517, 'k8s-worker-7': 13.554, 'k8s-worker-4': 18.616, 'k8s-worker-3': 8.446, 'k8s-worker-2': 23.601}, 'k8s-worker-3': {'k8s-worker-3': None, 'k8s-worker-5': 4.674, 'k8s-worker-9': 3.559, 'k8s-worker-4': 2.6809999999999996, 'k8s-worker-7': 3.5500000000000003, 'k8s-worker-2': 9.646999999999998, 'k8s-worker-1': 0.5670000000000001, 'k8s-worker-8': 16.895, 'k8s-worker-6': 8.507000000000001}, 'k8s-worker-9': {'k8s-worker-9': None, 'k8s-worker-3': 3.5660000000000003, 'k8s-worker-6': 15.681000000000001, 'k8s-worker-1': 3.666, 'k8s-worker-2': 11.542, 'k8

{'k8s-worker-1': None,
 'k8s-worker-9': 3.646,
 'k8s-worker-3': 0.49700000000000005,
 'k8s-worker-7': 6.472,
 'k8s-worker-6': 7.550000000000001,
 'k8s-worker-5': 5.472,
 'k8s-worker-2': 5.599,
 'k8s-worker-4': 11.704,
 'k8s-worker-8': 30.546}

In [2]:
raw_nodes = gather_all_nodes()  # returns raw K8s node objects

In [3]:
node_infos = build_nodeinfo_objects(raw_nodes)
for i in range(len(node_infos)):
    print(node_infos[i].node_name)
    print(node_infos[i].cpu_capacity)
    print(node_infos[i].current_cpu_usage)
    print(node_infos[i].mem_capacity)
    print(node_infos[i].current_mem_usage)
    print(node_infos[i].network_latency)
    print(node_infos[i].network_bandwidth)

k8s-master
32.0
0.6
128806.34765625
5748.34
{}
{}
k8s-worker-1
4.0
0.07
15988.421875
714.65
{'k8s-worker-9': 3.646, 'k8s-worker-3': 0.49700000000000005, 'k8s-worker-7': 6.472, 'k8s-worker-6': 7.550000000000001, 'k8s-worker-5': 5.472, 'k8s-worker-2': 5.599, 'k8s-worker-4': 11.704, 'k8s-worker-8': 30.546}
{'k8s-worker-3': '115 Mbits/sec', 'k8s-worker-8': '125 Mbits/sec', 'k8s-worker-9': '436 Mbits/sec', 'k8s-worker-5': '91.9 Mbits/sec', 'k8s-worker-2': '152 Mbits/sec', 'k8s-worker-6': '172 Mbits/sec', 'k8s-worker-7': '351 Mbits/sec', 'k8s-worker-4': '612 Mbits/sec'}
k8s-worker-2
4.0
0.18
15988.4296875
2161.45
{'k8s-worker-1': 5.597, 'k8s-worker-3': 9.58, 'k8s-worker-5': 11.647, 'k8s-worker-9': 11.917, 'k8s-worker-6': 23.654999999999998, 'k8s-worker-4': 12.67, 'k8s-worker-7': 16.677, 'k8s-worker-8': 39.675000000000004}
{'k8s-worker-1': '291 Mbits/sec', 'k8s-worker-3': '322 Mbits/sec', 'k8s-worker-8': '192 Mbits/sec', 'k8s-worker-6': '482 Mbits/sec', 'k8s-worker-9': '158 Mbits/sec', 'k8s-w

In [4]:
raw_pods = gather_all_pods(namespace='social-network')    # returns raw K8s pod objects
pod_infos  = build_podinfo_objects(raw_pods)

In [5]:
for i in range(len(pod_infos)):
    print(pod_infos[i].pod_name)
    print(pod_infos[i].cpu_req)
    print(pod_infos[i].mem_req)
    print(pod_infos[i].deployment_name)
    print(pod_infos[i].sla_latency_requirement)

compose-post-service-79fffc8f65-tvnvd
0.31
104.0
compose-post-service
200.0
home-timeline-redis-74777776d8-79l77
0.16
104.0
home-timeline-redis
200.0
home-timeline-service-5847cc645f-sn5x5
0.16
104.0
home-timeline-service
200.0
jaeger-7d4cb8c8db-lcz5j
0.16
104.0
jaeger
200.0
media-frontend-6f8b6db9bb-2bpb5
0.16
104.0
media-frontend
200.0
media-memcached-5584c4bbdf-z96sv
0.16
104.0
media-memcached
200.0
media-mongodb-6fd7768d66-9kqg6
0.16
104.0
media-mongodb
200.0
media-service-7fbfc5f4c-gfbdk
0.16
104.0
media-service
200.0
nginx-thrift-7f77f9f9df-b5tww
0.16
104.0
nginx-thrift
200.0
post-storage-memcached-786b6455f-tkbg2
0.16
104.0
post-storage-memcached
200.0
post-storage-mongodb-6cd66b64db-dv9rv
0.16
104.0
post-storage-mongodb
200.0
post-storage-service-696b99f894-486sz
0.16
104.0
post-storage-service
200.0
social-graph-mongodb-7db6d5b58-jn2bk
0.16
104.0
social-graph-mongodb
200.0
social-graph-redis-6b4dcf589-hgqs6
0.16
104.0
social-graph-redis
200.0
social-graph-service-7f767545d4-vr

In [13]:
# from platform import node



    

k8s-master
32.0
0.0
128806.34765625
0.0
{}
{}
k8s-worker-1
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-2
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-3
4.0
0.0
15988.421875
0.0
{}
{}
k8s-worker-4
4.0
0.0
15988.421875
0.0
{}
{}
k8s-worker-5
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-6
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-7
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-8
4.0
0.0
15988.4296875
0.0
{}
{}
k8s-worker-9
4.0
0.0
15988.4296875
0.0
{}
{}


In [4]:
raw_nodes

[{'api_version': None,
  'kind': None,
  'metadata': {'annotations': {'csi.volume.kubernetes.io/nodeid': '{"csi.tigera.io":"k8s-master"}',
                               'kubeadm.alpha.kubernetes.io/cri-socket': 'unix:///var/run/crio/crio.sock',
                               'node.alpha.kubernetes.io/ttl': '0',
                               'projectcalico.org/IPv4Address': '172.26.128.228/20',
                               'projectcalico.org/IPv4VXLANTunnelAddr': '192.168.235.192',
                               'volumes.kubernetes.io/controller-managed-attach-detach': 'true'},
               'creation_timestamp': datetime.datetime(2023, 12, 1, 10, 15, 17, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': None,
               'labels': {'beta.kubernetes.io/arch': 'amd64',
                          'beta.kubernetes.io/

In [14]:
from typing import Tuple
from prometheus_api_client import PrometheusConnect
from datetime import datetime, timedelta
import math

def fetch_live_node_usage_prometheus(node_name: str,
                                     prom_url: str = "http://localhost:9090") -> Tuple[float, float]: # NEED to specify the node name and prometheus urls
    """
    Fetch approximate CPU usage (in cores) and memory usage (in MiB) for a node
    from Prometheus using typical node_exporter metrics. 
    This is just an EXAMPLE. You must adjust your query and label matching 
    to match how your metrics are actually labeled!
    """
    prom = PrometheusConnect(url=prom_url, disable_ssl=True)

    # 1. Build a time range for the query. We query the instantaneous rate at "now"
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=5)  # 5-min window

    # 2. CPU Usage Query
    #
    # Here’s a typical pattern:
    #   100 * (1 - avg by(instance) (rate(node_cpu_seconds_total{mode="idle",instance="<node>:..."}[5m])))
    # if using IP addresses, you can use "instance=~'<Node_ip>:.*'"
    # if using node names, you can use "node=~'<node_host_name>:.*'"
    # That yields a percentage of CPU usage. We'll convert it to fractional cores 
    # by multiplying by the node’s CPU count or by using a different formula.
    #
    # Alternatively, if you want total CPU usage in cores:
    #   sum by (instance) (rate(node_cpu_seconds_total{mode!="idle",instance="<node>"}[5m]))
    #
    # For demonstration, let's do the sum of all non-idle cores usage:
    cpu_query = f'''
      sum by (instance) (
        rate(node_cpu_seconds_total{{mode!="idle", node=~"{node_name}.*"}}[5m])
      )
    '''
    # The instance label often looks like "ip:9100" or "node_name:9100". Adjust as needed.

    cpu_data = prom.custom_query_range(
        query=cpu_query,
        start_time=start_time,
        end_time=end_time,
        step="30s" # 30s resolution
    )

    cpu_usage_cores = 0.0
    if cpu_data:
        # Usually, you'll get a list of results. Let's take the last value from the first result.
        values = cpu_data[0]['values']  # array of [timestamp, value]
        if values:
            # take the most recent data point
            _, val_str = values[-1]
            cpu_usage_cores = float(val_str)  # e.g. "2.3" -> 2.3 cores

    # 3. Memory Usage Query
    #
    # A typical node_exporter metric is node_memory_MemTotal_bytes and node_memory_MemAvailable_bytes.
    # You can compute used = total - available. We'll do that for the node, then convert to MiB.
    mem_query = f'''
      node_memory_MemTotal_bytes{{node=~"{node_name}.*"}}
      -
      node_memory_MemAvailable_bytes{{node=~"{node_name}.*"}}
    '''
    mem_data = prom.custom_query_range(
        query=mem_query,
        start_time=start_time,
        end_time=end_time,
        step="30s"
    )

    mem_usage_mib = 0.0
    if mem_data:
        values = mem_data[0]['values']
        if values:
            _, val_str = values[-1]
            used_bytes = float(val_str)
            mem_usage_mib = used_bytes / (1024.0 * 1024.0)

    return (cpu_usage_cores, mem_usage_mib)

node_name = "k8s-worker-2"
prom_url = "http://10.105.116.175:9090"
cpu_usage, mem_usage = fetch_live_node_usage_prometheus(node_name, prom_url=prom_url)
print(f"Node {node_name} => CPU usage: {cpu_usage:.2f} cores, Memory usage: {mem_usage:.1f} MiB")

Node k8s-worker-2 => CPU usage: 0.12 cores, Memory usage: 1931.4 MiB


Prometheus URL found: http://10.105.116.175:9090
